<a href="https://colab.research.google.com/github/adammoss/MLiS2/blob/master/examples/rnn/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import itertools
import operator
import numpy as np
import nltk
import sys
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib inline

Download NLTK data

In [0]:
%%capture
nltk.download("book")

Upload deep_learning_sentences.txt file (or another file containing a list of sentences if you wish)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving deep_learning_sentences.txt to deep_learning_sentences.txt


Add sentence start and end tags, convert to lower case and strip newlines

In [0]:
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"

In [0]:
with open('deep_learning_sentences.txt', 'r') as f:
  sentences = f.readlines()
sentences = ["%s %s %s" % (sentence_start_token, x.rstrip('\n'), sentence_end_token) for x in sentences]

In [25]:
print("Parsed %d sentences." % (len(sentences)))
print("Example: %s" % sentences[2])

Parsed 7674 sentences.
Example: SENTENCE_START By adding more layers and more units within a layer, a deep network can represent functions of increasing complexity. SENTENCE_END


Tokenize the sentences into words

In [0]:
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]

In [27]:
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print("Found %d unique words tokens." % len(word_freq.items()))

Found 14998 unique words tokens.


In [0]:
VOCAB_SIZE = 8000

In [0]:
vocab = word_freq.most_common(VOCAB_SIZE-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])

Replace all words not in our vocabulary with the unknown token

In [0]:
unknown_token = "UNKNOWN_TOKEN"

In [0]:
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]

In [34]:
print("Example: %s" % tokenized_sentences[2])

Example: ['SENTENCE_START', 'By', 'adding', 'more', 'layers', 'and', 'more', 'units', 'within', 'a', 'layer', ',', 'a', 'deep', 'network', 'can', 'represent', 'functions', 'of', 'increasing', 'complexity', '.', 'SENTENCE_END']


Create the training data

In [0]:
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])

In [0]:
class RNN:
    
    def __init__(self, word_dim, hidden_dim=100):
        # Assign instance variables
        self.word_dim = word_dim
        self.hidden_dim = hidden_dim
        # Randomly initialize the network parameters
        self.U = np.random.uniform(-np.sqrt(1./word_dim), np.sqrt(1./word_dim), (word_dim, hidden_dim, ))
        self.V = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, word_dim))
        self.W = np.random.uniform(-np.sqrt(1./hidden_dim), np.sqrt(1./hidden_dim), (hidden_dim, hidden_dim))